In [1]:
import pandas as pd
import pulp as pl
import itertools
import numpy as np
import openpyxl
import warnings

# To ignore all warnings
warnings.filterwarnings("ignore")

# Preparacion de Datos
Primero vamos a agrupar nuestro dataset de demanda generado por los modelos de pasajeros y productos. Esto para que tengamos la información de cuando fue la última vez que un avion pasó por un aeropuerto por el cual pudo reabastecerse de productos. Por ejemplo, si un avion viaja Monterrey-Chihuahua-Tuxtla-CDMX entonces tenemos que considerar que el recargo de Monterrey tiene que llevar lo de Chihuahua y Tuxtla.

In [2]:
#Leyendo dataset
flight_df = pd.read_csv(r'..\..\data\Dummy_results_Jan01_v2.csv').drop(columns=['Unnamed: 0'])
flight_df['LastLoadChance'] = flight_df['DepartureStation']

#TODO: Esto es una fuerza bruta pero funciona pero se podria optimizar
#Generando la columna 'LastLoadChance' con el ultimo aeropuerto al cual se puede cargar cajas
aeropuertos = [ 'AW',    #'Monterrey',
                'AT',    #'AICM',
                'AO',    #'Cancún',
                'AK',    #'Guadalajara',
                'BM',    #'Tijuana',
                'BA',    #'Mérida',
                'AU']   #'Guanajuato' 

flight_dict = dict()
for a in flight_df['Aeronave'].unique():
    itinerario = flight_df[flight_df['Aeronave'] == a]
    itinerario = itinerario.sort_values(by='STD').reset_index(drop='true')
    
    last_important_port = 'overnight'
    
    for i in range(len(itinerario)):
        if(itinerario.loc[i]['DepartureStation'] in (aeropuertos)):
            last_important_port = itinerario.loc[i]['DepartureStation'] + ' - ' + itinerario.loc[i]['Flight_ID']

        itinerario.loc[i, 'LastLoadChance'] = last_important_port
    
    flight_dict[a] = itinerario


flight_df = pd.concat([flight_dict[a] for a in flight_df['Aeronave'].unique()], ignore_index=True)

#Agrupando dataframe por 'LastLoadChance' y nave
flight_df = flight_df.groupby(by=['Aeronave', 'LastLoadChance']).sum(numeric_only=True).reset_index()
flight_df.head()

,Aeronave,LastLoadChance,Capacity,Passengers,Bookings,AGUA NATURAL 600 ML,AMSTEL ULTRA,ARANDANO,ARANDANO MANGO MIX,ARCOIRIS,...,TOPOCHICO SELTZER FRESA-GUAYABA,TOPOCHICO SELTZER MANGO,TOSTITOS,TOSTITOS NACHOS CON DIP,ULTRA SELTZER FRAMBUESA,VINO BLANCO CRIA CUERVOS,VINO TINTO CRIA CUERVOS,VINO TINTO SANGRE DE TORO,XX LAGER,XX ULTRA
0,9H-SWB,AW - 026c3fe2baa82c273aacd8090782a2d2,356,227,320,2,0,4,6,0,...,2,0,4,0,4,2,0,0,0,2
1,9H-SWB,AW - 102d8c6a8b7a41404146ff802a5906d6,356,266,285,2,0,4,6,0,...,2,0,4,0,4,2,0,0,0,2
2,9H-SWB,AW - 7ec10a8cb9db5a5b9a587ae61c51113a,178,128,159,1,0,2,3,0,...,1,0,2,0,2,1,0,0,0,1
3,9H-SWB,BM - 92e240d6a4f155d53a1c457c3216c20b,178,119,156,1,0,2,3,0,...,1,0,2,0,2,1,0,0,0,1
4,9H-SWJ,AK - a1a603a1d338e2f556aaa1a7113f980f,178,146,168,1,0,2,3,0,...,1,0,2,0,2,1,0,0,0,1


# Modelo de Optimización
## Definicion de parámetros de entrada
Vamos a traer varios parametros pero principalmente son:

- Tabla de ``ProductoXCaja`` la cual contiene todas nuestras opciones de cajas que podemos agregar al avion con los productos que estas contienen. 
- ``Demanda`` la cual es nuestra tabla agrupada generada en el preprocesamiento con la cantidad de cada producto que necesitaremos por vuelo. 
- ``LimiteCajas`` lo que es cuantas cajas como máximo caben en un avion. 

In [3]:
#Parametros
tipos = list(range(1,29)) # lista los diferentes 28 tipos de cajas que existen
vuelos = flight_df['LastLoadChance'].unique()
LimiteCajas = 1000

ProductoXCaja = pd.read_csv(r'..\..\data\Tablasoptimizacion.csv')
Demanda = pd.melt(flight_df.drop(columns=['Aeronave', 'Capacity', 'Passengers', 'Bookings']),
                  id_vars=['LastLoadChance'],
                  var_name='producto')

productos = list(set(Demanda['producto']).intersection(set(ProductoXCaja['producto'])))

#filtrando datos faltantes
Demanda = Demanda[Demanda['producto'].isin(productos)]
ProductoXCaja = ProductoXCaja[ProductoXCaja['producto'].isin(productos)]

all_combinations = pd.MultiIndex.from_product([productos, tipos], names=['producto', 'tipo'])
ProductoXCaja = ProductoXCaja.set_index(['producto', 'tipo'])
ProductoXCaja = ProductoXCaja.reindex(all_combinations, fill_value=0)
Demanda = Demanda.set_index(['LastLoadChance', 'producto'])

## Planteamiento del problema

Teniendo esto en cuenta, haremos una optimización multiobjetivo.

### Variables de Decision

$$
\text{Sea     }R_v = \begin{cases} 
1 & \text{Si el vuelo $v$ se reabastece de productos}, \\
0 & \text{De otra forma}.
\end{cases}
$$

- Sea $C_{t,v}$ la cantidad de cajas del tipo $t$ que se agregan antes del vuelo $v$. 

### Restricciones iniciales

- No se puede cargar cajas en aviones si no se agrega la variable de reabastecimiento. 
$$R_v * 2^{30} \geq \sum_t C_{t,v} \quad \quad \forall\ v$$

- Nuestra demanda debe ser satisfecha siempre.
$$ProductoEnCaja_{p,t} * C_{t,v} \geq Demanda_{p,v} \quad \quad \forall \ p,v$$

- Tenemos un límite de cajas que podemos agregar al avión.
$$\sum_t C_{t,v} \leq LimiteCajas$$

In [4]:
prob = pl.LpProblem("LexicographicOptimization", pl.LpMinimize)

#Variables de decision
C = pl.LpVariable.dicts('C', list(itertools.product(tipos, vuelos)), lowBound=0, upBound=1000, cat='Integer')
R = pl.LpVariable.dicts('R', vuelos, lowBound=0, upBound=4, cat='Binary')
    
#* Satisfacemos la demanda estimada
for (p,v) in list(itertools.product(productos, vuelos)):
    prob += pl.lpSum([ProductoXCaja.loc[p, t][0] * C[(t, v)] for t in tipos]) >= Demanda.loc[(v, p)]['value']

#* No ingresamos más cajas de las que caben en el avion
for v in vuelos:
    prob += pl.lpSum([C[(t,v)] for t in tipos]) <= LimiteCajas
    
#* No cargar cajas cuando no hay recarga programada
# Utilizamos 2**30-1 como un numero gigante al cual nunca vamos a llegar de cajas recargadas
for v in vuelos:
    prob += R[(v)] * 2**30-1 >= pl.lpSum([C[(t, v)] for t in tipos])

## Funcion objetivo primaria

Tenemos como principal función objetivo minimizar la diferencia que hay entre cada producto abastecido en el avion y su demanda (el desperdicio o las sobras). 

1) $$\text{Min      } \sum_v \sum_p ProductoEnCaja_{p,t} * C_{t,v} - Demanda_{p,v}$$

Tambien aqui resolveremos el problema y guardaremos la solución la cual necesitaremos para la segunda optimización. 

In [5]:
prob += pl.lpSum([ProductoXCaja.loc[p,t][0] * C[(t,v)] for (p,t,v) in list(itertools.product(productos, tipos, vuelos))])\
        - pl.lpSum([Demanda.loc[v, p]['value'] for p,v in list(itertools.product(productos, vuelos))])

prob.solve()
primary_obj = pl.value(prob.objective)

## Optimización multiobjetivo
Vamos a anadir una nueva restricción auxiliar para prevenir que nuestro desperdicio incremente en más de 5% al hacer nuestra siguiente optimización en la cual minimizamos la cantidad de veces que se rabastecen los aviones ya que este es un proceso tardado y costoso. 

2) $$\text{Min      } \sum_v R_{v}$$

A este tipo de optimización se le llama optimización lexicográfica.

In [6]:
#Añadir que solo podamos deviar 5% del minimo desperdicio
prob += pl.lpSum([ProductoXCaja.loc[p,t][0] * C[(t,v)] for (p,t,v) in list(itertools.product(productos, tipos, vuelos))])\
        - pl.lpSum([Demanda.loc[v, p]['value'] for p,v in list(itertools.product(productos, vuelos))])\
        <= primary_obj * 1.05

#Nueva funcion objetivo
prob.setObjective( pl.lpSum([R[v] for v in vuelos]))

prob.solve()

#%%Imprimiendo 
print(f'Estatus de la solucion: {pl.LpStatus[prob.status]}')

Estatus de la solucion: Optimal


# Desplegando los resultados
Cantidad de cajas a ingresar por vuelo

In [7]:
df = pd.DataFrame(list(C.items()), columns=['Key', 'Value'])

df[['Tipo de Cajon', 'Vuelo']] = pd.DataFrame(df['Key'].tolist(), index=df.index)
df.drop(columns=['Key'], inplace=True)
df['Cantidad'] = df['Value'].apply(lambda x: x.varValue)
df = df.drop(columns='Value')
df = df.pivot(index='Vuelo', columns='Tipo de Cajon', values='Cantidad')
df

Tipo de Cajon,1,2,3,4,5,6,7,8,9,10,...,19,20,21,22,23,24,25,26,27,28
Vuelo,,,,,,,,,,,,,,,,,,,,,
AK - 15e2be3a9446058d56435385e99e2893,0.0,0.0,0.0,0.0,1.0,2.0,1.0,6.0,3.0,0.0,...,0.0,0.0,1.0,0.0,3.0,1.0,2.0,0.0,1.0,0.0
AK - 23bb6fdae9eefe920fad473e4d01c48f,0.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,2.0,0.0,...,0.0,0.0,1.0,0.0,2.0,1.0,1.0,0.0,1.0,0.0
AK - 2596756d2d85ce06072070d986911889,0.0,1.0,0.0,0.0,1.0,1.0,0.0,3.0,2.0,0.0,...,0.0,0.0,1.0,0.0,2.0,1.0,1.0,0.0,1.0,0.0
AK - 2f5df9fe48faabb7dda11f2aac6f15c6,0.0,1.0,3.0,0.0,0.0,1.0,0.0,0.0,2.0,1.0,...,0.0,0.0,1.0,0.0,2.0,1.0,1.0,0.0,1.0,0.0
AK - 38bfd14b2446f2ad7f37214b6b8554d4,0.0,1.0,3.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,...,0.0,0.0,1.0,0.0,2.0,1.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BM - e3505b065fd040f9339430c83e9d4af4,4.0,1.0,12.0,6.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,6.0,1.0,3.0,0.0,2.0,0.0
BM - f11de826fb0bae443b81be3e6039d0fb,0.0,1.0,12.0,6.0,0.0,4.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,6.0,1.0,3.0,0.0,2.0,0.0
BM - f1295a5ac10dd9c7c88eace07ab0f65d,1.0,1.0,3.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,1.0,0.0,2.0,1.0,0.0,0.0,1.0,1.0
